<a href="https://colab.research.google.com/github/HedersonSantos/age-related/blob/main/auto_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#!pip install h2o

In [14]:
#!wget https://raw.githubusercontent.com/HedersonSantos/age-related/main/tree_features\ .csv
#!mv tree_features\ .csv tree_features.csv

In [2]:
#https://medium.com/swlh/automating-machine-learning-a486d365e423#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6IjJkOWE1ZWY1YjEyNjIzYzkxNjcxYTcwOTNjYjMyMzMzM2NkMDdkMDkiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2ODQ4ODUzNDYsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMDg2MDE5ODExNjQ3MTEyMjk1MCIsImVtYWlsIjoiaGVkZXJzb24uc2FudG9zQGdtYWlsLmNvbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJhenAiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJuYW1lIjoiSGVkZXJzb24gU2FudG9zIiwicGljdHVyZSI6Imh0dHBzOi8vbGgzLmdvb2dsZXVzZXJjb250ZW50LmNvbS9hL0FHTm15eGJhZ3FGUGJhOWp5SW9iZE1HTnhlU1V3bmdrR01qS0xmbWpUTWhQPXM5Ni1jIiwiZ2l2ZW5fbmFtZSI6IkhlZGVyc29uIiwiZmFtaWx5X25hbWUiOiJTYW50b3MiLCJpYXQiOjE2ODQ4ODU2NDYsImV4cCI6MTY4NDg4OTI0NiwianRpIjoiMGFkNjIyNzNkNTIyNWQzY2EzZmU4NGZjNWMwZjJlOTI5OWJiMTA1MCJ9.QqawwPG8xv1JA7zziFxb41KMd8jkKkHXcWaoISmQlPWA4GFNNQPJB6NN8F9WwqDz8K7kn0hTmYKp8WSpG07mhcYj8hgCIgminz1cJOW1xiQUsvwcfLPO5mrienkdM1esC9MEpbQDWte1-Zm_cJ8V9i_MFFAgjLuSS8aBN0jIM2hmEjS-fazmMQ_A_jZN_vCK1Hjw9-puzXpe4YS-i3Wc3aiiiRBwVl48sMYrT9r8-sNkTTv6uQE2Y37lZ1AutjiQxJtVUIdTBI65sSTP-uEC46xtWi85UMXGXW9FqdBnZn8_h87MCnvhjFrDcjBwvn1syOuF3FSG_dUHoQVT4v3TxA
import pandas as pd


In [4]:
df = pd.read_csv('tree_features.csv', header=0)
df['Class']=df['Class'].apply(lambda x: 'S' if x==1 else 'N')
df['Class']=df['Class'].astype('category')
df.head(10)

,AB,AF,BC,BQ,CC,CR,DA,DE,DH,DI,DU,EE,EH,FD,FE,FI,FL,FR,GL,Class
0,0.209377,3109.03329,5.555634,152.707705,0.563481,0.069225,69.08340,295.570575,0.284232,89.245560,5.310690,1.987283,0.949104,10.265073,9028.291921,3.583450,7.298162,1.73855,0.120343,S
1,0.145282,978.76416,1.229900,14.754720,0.484710,1.117800,70.79836,178.553100,0.363489,110.581815,0.005518,0.858603,0.003042,0.296850,6785.003474,10.358927,0.173229,0.49706,21.978000,N
2,0.470030,2635.10654,1.229900,219.320160,0.495852,0.700350,70.81970,321.426625,0.210441,120.056438,1.289739,8.146651,0.377208,8.745201,8338.906181,11.626917,7.709560,0.97556,0.196941,N
3,0.252107,3819.65177,1.229900,11.050410,0.717882,0.636075,47.27586,196.607985,0.292431,139.824570,2.655345,3.813326,0.614484,7.884336,10965.766040,14.852022,6.122162,0.49706,0.155829,N
4,0.380297,3733.04844,102.151980,149.717165,0.536467,0.693150,74.06532,200.178160,0.207708,97.920120,1.144902,3.490846,0.164268,4.274640,16198.049590,13.666727,8.153058,48.50134,0.096614,S
5,0.209377,2615.81430,1.229900,16.526120,0.639460,0.857625,55.22404,135.489250,0.478275,135.317865,0.005518,2.394414,0.003042,0.296850,8517.278846,10.981896,0.173229,0.49706,21.978000,N
6,0.348249,1733.65412,1.229900,344.644105,0.946323,0.610950,19.21570,107.907985,0.460510,176.625563,1.006962,2.620150,0.097344,1.460502,3903.806766,10.777915,4.408484,0.86130,0.065096,N
7,0.269199,966.45483,1.229900,6.199900,0.771909,1.109625,63.21684,326.225295,0.325227,83.769368,2.117379,3.277203,0.292032,4.518057,18090.349450,10.342388,6.591896,0.49706,0.092873,N
8,0.346113,3238.43674,3.626448,61.642115,0.855496,1.050375,76.77356,231.134460,0.330693,131.349555,0.613833,1.342323,0.066924,1.389258,3380.026318,11.450501,4.762291,1.18262,0.073416,N
9,0.324748,5188.68207,1.229900,107.276505,0.718271,0.711712,31.17774,403.247940,0.351191,129.175267,0.005518,3.658132,0.003042,0.296850,3142.390734,12.329825,0.173229,1.57151,21.978000,N


In [5]:
X_train = df[:-1].values
y_train = df['Class']


In [6]:
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.automl import H2OAutoML


In [7]:
h2o.init()
health = h2o.H2OFrame(df)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.19" 2023-04-18; OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpgc506ax_
  JVM stdout: /tmp/tmpgc506ax_/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpgc506ax_/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,25 days
H2O_cluster_name:,H2O_from_python_unknownUser_j50luv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
# set the predictor names and the response column name
predictors = ['AB', 'AF', 'BC', 'BQ', 'CC', 'CR',
              'DA', 'DE', 'DH', 'DI', 'DU', 'EE',
              'EH', 'FD ', 'FE', 'FI', 'FL', 'FR', 'GL']
response = "Class"

# split into train and validation sets
train, valid = health.split_frame(ratios = [.90], seed = 1234)



In [9]:
aml = H2OAutoML(max_runtime_secs=60*120,
                seed=1,
                balance_classes=False,
                project_name='age_related'
)
%time aml.train(x=predictors, y=response, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 1min 14s, sys: 5.45 s, total: 1min 20s
Wall time: 1h 57min 54s


key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DeepLearning base models (used / total),1/1
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [10]:
lb = aml.leaderboard
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_6_AutoML_1_20230524_00249,0.968352,0.162328,0.898926,0.0833086,0.219214,0.0480549
GBM_lr_annealing_selection_AutoML_1_20230524_00249_select_model,0.964318,0.184779,0.883365,0.134466,0.235084,0.0552643
GBM_grid_1_AutoML_1_20230524_00249_model_10,0.963723,0.182844,0.880963,0.106533,0.234819,0.0551398
GBM_grid_1_AutoML_1_20230524_00249_model_64,0.963026,0.184701,0.869724,0.113128,0.236946,0.0561433
GBM_grid_1_AutoML_1_20230524_00249_model_15,0.963015,0.197733,0.869818,0.130032,0.245334,0.0601887
GBM_grid_1_AutoML_1_20230524_00249_model_4,0.962443,0.189725,0.863123,0.115345,0.241905,0.0585179
GBM_grid_1_AutoML_1_20230524_00249_model_95,0.96146,0.19063,0.852737,0.116454,0.242259,0.0586896
GBM_grid_1_AutoML_1_20230524_00249_model_61,0.961129,0.204767,0.876786,0.117174,0.244718,0.0598867
StackedEnsemble_AllModels_6_AutoML_1_20230524_00249,0.960694,0.17239,0.891106,0.0836972,0.222103,0.04933
GBM_grid_1_AutoML_1_20230524_00249_model_82,0.960477,0.190026,0.853264,0.114236,0.239707,0.0574594


In [11]:
#take the best performing model and find the variable which is important for the dependent variable.
se = aml.leader 
#Loading Stack Ensambled Metelearner model 
metalearner = h2o.get_model(se.metalearner()['name'])
metalearner.varimp()


/usr/local/lib/python3.10/dist-packages/h2o/estimators/stackedensemble.py:926: H2ODeprecationWarning: The usage of stacked_ensemble.metalearner()['name'] will be deprecated. Metalearner now returns the metalearner object. If you need to get the 'name' please use stacked_ensemble.metalearner().model_id
  warnings.warn(


[('DeepLearning_grid_1_AutoML_1_20230524_00249_model_8',
  1.81365966796875,
  1.0,
  0.5203963294550645),
 ('GBM_lr_annealing_selection_AutoML_1_20230524_00249_select_model',
  1.5365978479385376,
  0.8472360471352852,
  0.44089852911122046),
 ('XGBoost_lr_search_selection_AutoML_1_20230524_00249_select_grid_model_4',
  0.1348932534456253,
  0.07437627677782685,
  0.03870514143371505),
 ('XRT_1_AutoML_1_20230524_00249', 0.0, 0.0, 0.0),
 ('DRF_1_AutoML_1_20230524_00249', 0.0, 0.0, 0.0),
 ('GLM_1_AutoML_1_20230524_00249', 0.0, 0.0, 0.0)]

In [ ]:
#validação
model = h2o.get_model('colocar nome do modelo de melhor desempenho')
model.model_performance(valid)

In [ ]:
#importância das variáveis
model.varimp_plot(num_of_features=3)

In [ ]:
#dependência da variável com a classe
model.partial_plot(train, cols=["nome da variável de maior importância"], figsize=(5,5));

In [12]:
#salva o modelo
model_path = h2o.save_model(model=model, path='best_model',    
                      force=True)

NameError: ignored